In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS

In [3]:
data = pd.read_csv("artists_df.csv")
data = data[data['wiki'].apply(lambda x: isinstance(x, str))]
descriptions = data["wiki"].tolist()

descriptions = [el[:5000] for el in descriptions]
data["wiki"] = descriptions

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedding_model.encode(descriptions)

metadata = [{"artist": row['artist'], "description": row["description"], "wiki": row["wiki"]} for _, row in data.iterrows()]
text_embeddings = zip(descriptions, embeddings)

vector_store = FAISS.from_embeddings(text_embeddings, embedding_model, metadata)

/home/egorbl/programming/artrag/venv/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [ ]:
from rag import ArtAssistant

base_url = ""
api_key = ""

ass = ArtAssistant(vector_store, embedding_model, base_url, api_key, "gpt-3.5-turbo", 1)

In [5]:
ass.handle_user_query("каких кубистов ты знаешь. с датами рождения")

'К сожалению, в предоставленной информации приводится информация только об одном кубисте - Жорже Браке (Georges Braque), родившемся 13 мая 1882 года.'